In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
 ls data

shoes_prices.csv


In [0]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)

In [0]:
 def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [29]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [30]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
str_dict = df.features.head().values

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)


In [184]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [185]:
 def get_name_feat(key):
   return 'feat_' + key
 
 for key in tqdm_notebook(keys):
   df[get_name_feat(key)] =  df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan)

In [48]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_used with equipment type', 'feat_fits brands',
       'feat_leg opening:', 'feat_alarm', 'feat_international shipping?',
       'feat_chronograph', 'feat_fit:', 'feat_material type',
       'feat_lens socket width', 'feat_quantity in set'],
      dtype='object', length=526)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [120]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

feats_cat = [x for x in df.columns if '_cat' in x and x != 'feat_catalog'] 

In [174]:
df['brand'] = df['brand'].map(lambda brand: str(brand).lower())
df[ df.brand == df.feat_brand].shape

(8846, 1002)

In [175]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_adjustable_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats, model)

(-57.292477059111135, 4.044146736895246)

In [176]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(perm, feature_names=feats)


Weight,Feature
0.2610 ± 0.0126,brand_cat
0.1193 ± 0.0110,feat_material_cat
0.0297 ± 0.0028,feat_gender_cat
0.0251 ± 0.0033,feat_adjustable_cat
0.0213 ± 0.0014,feat_brand_cat


In [186]:
df['feat_material type'].value_counts()

synthetic    6
suede        1
Name: feat_material type, dtype: int64

In [0]:
df['feat_material type'] = df['feat_material type'].map(lambda x: 'leather' if 'leather' in str(x) else x)

In [107]:
df[False == df.weight.isnull()].shape

(570, 1003)

In [130]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_used with equipment type_cat', 'feat_fits brands_cat',
       'feat_leg opening:_cat', 'feat_alarm_cat',
       'feat_international shipping?_cat', 'feat_chronograph_cat',
       'feat_fit:_cat', 'feat_material type_cat', 'feat_lens socket width_cat',
       'feat_quantity in set_cat'],
      dtype='object', length=1001)

In [187]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   matrix_1/day4.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	matrix_1/day5.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
g